In [ ]:
import numpy as np
import pickle
import csv
import os
import math
import random
import pandas as pd
from gensim.models import Word2Vec
import multiprocessing as mp

# Load in Healthcare articles
csv_file_path = "intermediate/local_housing.csv"
df = pd.read_csv(csv_file_path)
eras = range(2000, 2023)
counties = df['Publisher'].unique()

def process_county(county):
    data = []
    for era in eras:
        filtered_df = df.dropna(subset=['Year'])
        condition = (filtered_df['Year'] == era) & (filtered_df['Publisher'] == county)
        county_df = filtered_df.loc[condition, 'Text']
        x = '\n'.join(county_df.astype(str))
        data.extend(x.split('\n'))
    return data

# Parallelize the processing
num_cores = mp.cpu_count()
pool = mp.Pool(num_cores)

data = pool.map(process_county, counties)
pool.close()
pool.join()

list_of_lists = ([sentence.split() for sentence in county_data] for county_data in data)

with open('intermediate/housing_corpus.pickle', 'wb') as f:
    pickle.dump(list_of_lists, f)

full_corpus = [item for sublist in list_of_lists for item in sublist]

# Train Word2Vec model
start_model = Word2Vec(full_corpus, vector_size=300, min_count=100, epochs=10, sg=1, hs=0, negative=5, window=6, workers=num_cores-1)

model_name = "intermediate/housing.model"
start_model.save(model_name)


In [ ]:
processed_data

In [ ]:
full_corpus = []
flatten = [item for sublist in list_of_lists for item in sublist]
full_corpus.extend(flatten)

In [ ]:
num_cores = mp.cpu_count()

# Ran Apr 3 5:16 PM 

start_model = Word2Vec(full_corpus, vector_size = 300, min_count = 100, epochs = 10, sg = 1, hs = 0, negative = 5, window = 6, workers = num_cores-1)

model_name = "intermediate/housing.model"

start_model.save(model_name)